# Fast Data Pipeline with Threading

The GIL means that threads are not always helpful in Python, but there are two special cases where Python threading can be helpful:
* I/O operations can occur without blocking the GIL, and
* Calls to C++/Rust libraries (eg, many librosa operations) release the GIL while doing the heavy lifting.

In both of these cases, we can improve execution speed by parallelizing effectively. 

This is particularly important for *data pipelines*. When training a model, we typically need to feed it subsets of our (gigantic, too big to fit in memory) dataset. Without threading, we need to load the next batch of data, wait for the model to evaluate and update, then load the next batch of data, and so on. This means that our fancy GPU is going to waste time while we load the data!

Instead, we can load data *while* the model is running, and (hopefully) have the next batch of data ready and waiting. This greatly improves throughput. At the same time, we can make the data loader multi-threaded, to load data from more files in parallel (ie, faster).

Let's start with a very simple example of how to use a `ThreadPoolExecutor`.

To use the executor:
1) Set it up, with some number of `max_workers`.
2) Use `executor.submit(fn, arg1, arg2, ...)` to give the workers some work. The worker will run the submitted function, and evaluate it with the provided arguments - `fn(arg1, arg2, ...)`. This submit call returns a *future* object, which you should store for later to get the results.
3) Get results from the future objects.

Here's some example code to get you started.

In [1]:
from concurrent.futures import ThreadPoolExecutor

# The function we want the workers to evaluate.
fun = lambda a: a**2

# Create an `executor` that we can pass work to.
with ThreadPoolExecutor(max_workers=2) as executor:
  results = []
  for i in range(10):
    # Each result is a `future` object, which can tell us whether
    # it is still running, and eventually provide a result.
    results.append(executor.submit(fun, i))
  results = [r.result() for r in results]
  print(results)

# We can also use a `map` approach more concisely.
with ThreadPoolExecutor(max_workers=2) as executor:
  # map_results is a generator, yielding results as they are ready.
  map_results = executor.map(fun, range(10))
  print(list(map_results))


[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]
[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]


In [ ]:
import dataclasses
from typing import Generator
from etils import epath
import numpy as np
import librosa
import tqdm

@dataclasses.dataclass
class AudioExample:
  audio: np.ndarray
  file_id: str
  offset: float


In [ ]:
# ------------------------------------------------------------
# 📝 Exercise 0 – Plain Old Data Loader
# ------------------------------------------------------------
# First, write a data loader which loads audio data from a
# directory. The data loader should yield `AudioExample` objects
# with the following fields:
# - `audio`: the audio data as a numpy array
# - `file_id`: the file name of the audio file
# - `offset`: the offset in seconds of the audio file
# The data loader should load all audio files in the directory
# and yield them one by one. The audio files are in WAV format.
# ------------------------------------------------------------

def data_loader(
    base_path: str,
    file_glob: str, 
    window_size_s: float,
    sample_rate: int) -> Generator[AudioExample, None, None]:
  """Loads audio files from a directory and yields AudioExample objects."""
  raise NotImplementedError(
      'Implement the data_loader function to load audio files from a directory.'
  )

for x in tqdm.tqdm(data_loader(
    '/mnt/class_data/anuraset', 
    'raw_data/INCT17/*.wav', 
    5.0, 32000)):
  pass

In [ ]:
# ------------------------------------------------------------
# 📝 Exercise 1 – Threaded Old Data Loader
# ------------------------------------------------------------
# Next, write a threaded version of the data loader.
# ------------------------------------------------------------

def threaded_data_loader(
    base_path: str,
    file_glob: str, 
    window_size_s: float,
    sample_rate: int,
    max_workers: int = 2) -> Generator[AudioExample, None, None]:
  """Loads audio files from a directory and yields AudioExample objects."""
  raise NotImplementedError(
      'Implement the threaded_data_loader function to load audio files from a directory.'
  )

for x in tqdm.tqdm(threaded_data_loader(
    '/mnt/class_data/anuraset', 
    'raw_data/INCT17/*.wav', 
    5.0, 32000, max_workers=2)):
  pass

**Extra challenge:**
Our current data iterator is pretty good, but has a subtle failure mode: If we process the data slower than the iterator loads the data, we can wind up with an ever-growing backlog of loaded data, until we run out of memory. Refactor the data loader so that it only preloads a fixed number of examples at a time.